In [21]:
import ee
ee.Initialize(project='gist-itproject2')  


In [22]:
import ee, geemap
from ipyleaflet import Marker, DivIcon, Popup
from ipywidgets import HTML  # <-- correct source for HTML widget

ee.Initialize(project='gist-itproject2')

# ---------- 1) Your country profiles ----------
profiles = [
    {"Country": "Australia", "Capital": "Canberra",
     "RulingParty": "Australian Labor Party", "Opposition": "Liberal-Nationals Coalition",
     "Religion1": "No Religion (38.7%)", "Religion2": "Catholic (19.6%)"},
    {"Country": "China", "Capital": "Beijing",
     "RulingParty": "Chinese Communist Party (CCP)", "Opposition": "None (one-party state)",
     "Religion1": "Unaffiliated (52.1%)", "Religion2": "Chinese Folk Religion (21.9%)"},
    {"Country": "India", "Capital": "New Delhi",
     "RulingParty": "BJP-led coalition", "Opposition": "INDIA bloc",
     "Religion1": "Hinduism (79.8%)", "Religion2": "Islam (14.2%)"},
    {"Country": "Indonesia", "Capital": "Nusantara (transitional)",
     "RulingParty": "Gerindra (President)", "Opposition": "PDI-P (role in flux)",
     "Religion1": "Muslim (87.2%)", "Religion2": "Protestant (7%)"},
    {"Country": "Japan", "Capital": "Tokyo",
     "RulingParty": "LDP–Komeito coalition", "Opposition": "CDP",
     "Religion1": "Shinto (48.6%)", "Religion2": "Buddhism (46.4%)"},
    {"Country": "Malaysia", "Capital": "Kuala Lumpur",
     "RulingParty": "Pakatan Harapan (PH) coalition", "Opposition": "Perikatan Nasional (PN)",
     "Religion1": "Islam (63.5%)", "Religion2": "Buddhism (18.7%)"},
    {"Country": "New Zealand", "Capital": "Wellington",
     "RulingParty": "National Party–led coalition", "Opposition": "Labour Party",
     "Religion1": "No Religion (51.6%)", "Religion2": "Christianity (32.3%)"},
    {"Country": "Philippines", "Capital": "Manila",
     "RulingParty": "Alyansa para sa Bagong Pilipinas", "Opposition": "DuterTen coalition",
     "Religion1": "Roman Catholic (78.8%)", "Religion2": "Islam (6.4%)"},
    {"Country": "Korea, South", "Capital": "Seoul",
     "RulingParty": "People Power Party (PPP)", "Opposition": "Democratic Party (DP)",
     "Religion1": "Unaffiliated (46.5%)", "Religion2": "Christian (29%)"},
    {"Country": "Thailand", "Capital": "Bangkok",
     "RulingParty": "Pheu Thai coalition", "Opposition": "People's Party (formerly MFP)",
     "Religion1": "Theravada Buddhism (90–95%)", "Religion2": "Muslim (~5%)"},
    {"Country": "Vietnam", "Capital": "Hanoi",
     "RulingParty": "Communist Party of Vietnam (CPV)", "Opposition": "None (one-party)",
     "Religion1": "Irreligion/Folk Religion (76.5%)", "Religion2": "Buddhism (13.3%)"},
]

# ---------- 2) Build Indo-Pacific polygons with attributes ----------
world = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

def attach_profile(p):
    fc = world.filter(ee.Filter.eq('country_na', p["Country"]))
    return fc.map(lambda f: f.set(p))

ip_fc = ee.FeatureCollection([])
for p in profiles:
    ip_fc = ip_fc.merge(attach_profile(p))

extent = ee.Geometry.Rectangle([-70, -50, 30, 40], geodesic=False)
non_ip_fc = world.filter(ee.Filter.Not(ee.Filter.intersects('.geo', extent)))

# ---------- 3) Map setup with polygons ----------
m = geemap.Map(center=[-10, 140], zoom=3)
m.add_basemap("CartoDB.Positron")

non_ip_style = non_ip_fc.style(color="a0a0a0", fillColor="5ca2ff", width=0.2)
ip_style     = ip_fc.style(color="ffffff", fillColor="ffcc00", width=1.4)

m.addLayer(non_ip_style, {}, "Non–Indo-Pacific (blue)", True, 0.18)
m.addLayer(ip_style, {}, "Indo-Pacific (polygons)")

# ---------- 4) Add diamond markers with popups ----------
centroids = ip_fc.map(lambda f: ee.Feature(f.geometry().centroid(), f.toDictionary()))
gj = geemap.ee_to_geojson(centroids)

diamond_html = """
<div style="
  width:14px;height:14px;
  background:#ffcc00;
  border:2px solid white;
  transform: rotate(45deg);
  box-shadow: 0 0 3px rgba(0,0,0,0.35);
"></div>
"""

for feat in gj["features"]:
    lon, lat = feat["geometry"]["coordinates"]
    props = feat["properties"]
    popup_html = HTML(value=f"""
        <div style='font-family:system-ui; min-width:220px'>
          <b>{props.get('Country','')}</b><br/>
          <b>Capital:</b> {props.get('Capital','')}<br/>
          <b>Ruling:</b> {props.get('RulingParty','')}<br/>
          <b>Opposition:</b> {props.get('Opposition','')}<br/>
          <b>Religions:</b> {props.get('Religion1','')}, {props.get('Religion2','')}
        </div>
    """)
    popup = Popup(child=popup_html, max_width=260)
    marker = Marker(location=(lat, lon), icon=DivIcon(html=diamond_html), draggable=False)
    marker.popup = popup
    m.add_layer(marker)

m.addLayer(extent, {"color": "FF0000"}, "Study extent", False)
m.addLayerControl()
m

Map(center=[-10, 140], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…